# Parte 3: Olivetti Faces

1.	Seguirá trabajando con el dataset: Olivetti Faces. Como primer paso usted deberá cargar los datos y corroborar las dimensiones del dataset, el cual debe ser 400 x 4096

2. Sabiendo que el número de clusters (personas) es 40, aplique K-Means y cluster jerárquico. Mediante el método del codo, identifique cuántos clusters le entrega cada uno de los algoritmos. ¿Cuál le da mejor resultado?

3. Ahora aplique PCA al dataset. Quédese con un número de componentes principales tal que pueda capturar el 90% de la varianza explicada . ¿Cuántos componentes principales son?

4. Aplique K-Means y cluster jerárquico al dataset reducido por PCA. Mediante el método del codo, identifique cuántos clusters le entrega cada uno de los algoritmos. ¿Cuál algoritmo le da mejor resultado? Compare los resultados con y sin PCA. ¿Sirvió de algo aplicar PCA?

5. Divida el dataset en training y testing set. 80% training y 20% test. Use el coeficiente de Silhouette para estimar los valores de los parámetros de DBSCAN aplicado al training set.

6. Con los valores estimados, aplique DBSCAN al training set y luego haga predicciones al testing set. Compare que tan bien estimo las etiquetas el algoritmo con respecto a las verdaderas etiquetas del testing set.



Considerando lo aprendido en este módulo, implemente lo que Ud. crea necesario para mejorar los resultados del problema de decoding visto en clases 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming your dataset is stored in a DataFrame named 'df'
# Separate the dependent variable 'Status' from the independent variables 'X'
X = df_lexp.drop(columns=['Enc_status'])
Y = df_lexp['Enc_status']

# Impute missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to reduce the data to 2 components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the PCA results and add the 'Status' column
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Enc_status'] = Y

# Visualize the data in a scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Enc_status', data=pca_df, palette='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Visualization')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Split the PCA-transformed data into training and testing sets
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

# Initialize the KNN classifier
k = 5  # Number of neighbors to consider (can be adjusted)
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the model on the training data
knn_classifier.fit(X_train_pca, Y_train_pca)

# Predict 'Status' on the test set
Y_pred_pca = knn_classifier.predict(X_test_pca)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Combine the PCA-transformed data and predicted class labels
pca_result_df = pd.DataFrame(data=X_test_pca, columns=['PC1', 'PC2'])
pca_result_df['Predicted_Status'] = Y_pred_pca

# Visualize the data in a scatter plot with the predicted classes
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Predicted_Status', data=pca_result_df, palette='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('KNN Classification with PCA')
plt.show()